---
# --- Day 20: Pulse Propagation ---
---

In [4]:
from typing import List, Dict
import numpy as np

## Load data

In [5]:
full_puzzle_data = False

In [6]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day20_input{file_suffix}1.txt", "r") as f:
    data = f.read().splitlines()

In [7]:
data

['broadcaster -> a, b, c', '%a -> b', '%b -> c', '%c -> inv', '&inv -> a']

In [15]:
class CommunicationModule:
    
    def __init__(self, module_name: str, destinations: List[str]):
        self.module_name = module_name
        self.destination_modules = destinations
        self.pulses_sent = [0, 0]
        self.received_pulses = []
        self.pulse_to_transmit = None
        
    def receive_pulse(self, pulse: int, sender: str) -> None:
        self.received_pulses.append(pulse)
        first_received_pulse = self.received_pulses.pop(0)
        self._update_status(first_received_pulse, sender)
    
    """Interface for communications modules""" 
    def _update_status(self, pulse: int, sender: str) -> None:
        raise NotImplementedError
        
    def transmit(self, modules: Dict) -> None:
        if self.pulse_to_transmit is not None:
            for m in self.destination_modules:
                modules[m].receive_pulse(self.pulse_to_transmit, self.module_name)
                print(f"{self.module_name} {self.pulse_to_transmit} -> {m}")
            self.pulses_sent[self.pulse_to_transmit] += len(self.destination_modules)
            for m in self.destination_modules:
                modules[m].transmit(modules)

In [9]:
class BroadcasterModule(CommunicationModule):
    module_type = "b"
    
    def _update_status(self, pulse: int, sender: str) -> None:
        self.pulse_to_transmit = pulse

In [10]:
class FlipFlopModule(CommunicationModule):    
    module_type = "%"
    active = False
    
    def _update_status(self, pulse: int, sender: str) -> None:
        if pulse == 0:
            self.active = not self.active
            self.pulse_to_transmit = int(self.active)
        else:
            self.pulse_to_transmit = None

In [11]:
class ConjunctionModule(CommunicationModule):
    module_type = "&"
    memory = dict()
    
    def _update_status(self, pulse: int, sender: str) -> None:
        self.memory[sender] = pulse
        self.pulse_to_transmit = 0 if all([v==1 for v in self.memory.values()]) else 1 

In [12]:
def instantiate_modules(file_tag: str) -> Dict[str, CommunicationModule]:
    with open(f"data/day20_{file_tag}.txt", "r") as f:
        data = f.read().splitlines()
    modules = []
    for row in data:
        module_string, dests = row.split(" -> ")
        destinations = dests.split(", ")
        module_type = module_string[0]
        if module_type == "b":
            modules.append(BroadcasterModule("broadcaster", destinations))
        elif module_type == "%":
            modules.append(FlipFlopModule(module_string[1:], destinations))
        elif module_type == "&":
            modules.append(ConjunctionModule(module_string[1:], destinations))
        else:
            raise ValueError("Module type unknown.")
    modules_dict = {m.module_name: m for m in modules}
    # create input connections for conjunction modules
    for k, m in modules_dict.items():
        for d in destinations:
            if modules_dict[d].module_type == "&":
                modules_dict[d].memory.update({k: 0})
    return modules_dict

In [13]:
modules = instantiate_modules("input_test1")

## --- Part One ---

In [14]:
modules["broadcaster"].receive_pulse(0, "button")
modules["broadcaster"].transmit(modules)

broadcaster 0 -> a
[0]
broadcaster 0 -> b
broadcaster 0 -> c
[1]
a 1 -> b
c 1 -> inv
inv 0 -> a
[0]
a 0 -> b
b 0 -> c
c 0 -> inv
inv 1 -> a


In [219]:
np.sum([m.pulses_sent for m in modules.values()], axis=0) + np.array([1, 0])

array([8, 3])

In [211]:
[m.pulses_sent for m in modules.values()]

[[3, 0], [1, 1], [1, 0], [1, 1], [1, 1]]

## --- Part Two ---